# Create neighborhood boundary polygon feature class with the size of "residential area"

In [3]:
import arcpy
from arcpy import env
import os
try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    # Set local variables
    input_gdb = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    clipFC = os.path.join(input_gdb,"LandNoLakeForest")
    # Get a Residential area for census data
    arcpy.Clip_analysis('Census2010_Shimane', clipFC,"Census2010_Shimane_ResArea")
    
    # Calculate area
    inTable = "Census2010_Shimane_ResArea"
    fieldName1 = "AreaRes"
    expression1 = "!shape.area@SQUAREKILOMETERS!"
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    # Execute CalculateField - Get a area size in square kilometers
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")

    # Join the AreaRes field to original data
    inFeatures = "Census2010_Shimane"
    joinField1 = "OBJECTID"
    joinTable = "Census2010_Shimane_ResArea"
    joinField2 = "OBJECTID"
    fieldList = ["AreaRes"] 
    outFeatures = "Census2010_Shimane2"
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


## Create walkability index - Neighborhood level
### 1. Residential density (already in the neighborhood data)
### 2. Bus stop density (spatial join with bus stop point data 2010)
### 3. Intersection density (spatial join with intersection with >= 3 legs point data 2012)
### 4. Slope (spatial join with mean slope 5 mesh data 2015)
### 5. Age-specific population density (already in the neighborhood data)

In [7]:
### Import system modules
import arcpy
from arcpy import env
import os

try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    ### Set local variables
    targetFeatures1 = ("Census2010_Shimane2")
    targetFeatures2 = ("Census2010_Shimane2Bus")
    targetFeatures3 = ("Census2010_Shimane2BusInter")
    joinFeatures1 = ("busstop_Chugoku2010")
    joinFeatures2 = ("Intersect_Shimane2012")
    joinFeatures3 = ("SlopeShimane2")
    outfc1 = ("Census2010_Shimane2Bus")
    outfc2 = ("Census2010_Shimane2BusInter")
    outfc3 = ("Census2010_Shimane2BusInterSlope")
    
    ### Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures1, joinFeatures1, outfc1, "JOIN_ONE_TO_ONE", "KEEP_ALL", "","COMPLETELY_CONTAINS")
    arcpy.SpatialJoin_analysis(targetFeatures2, joinFeatures2, outfc2, "JOIN_ONE_TO_ONE", "KEEP_ALL", "","COMPLETELY_CONTAINS")
    
    ### Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures3)
    fieldmappings.addTable(joinFeatures3)
    # First get the slope fieldmap. slope is a field in the SlopeShimane2 feature class.
    # The output will have the attributes of the SlopeShimane2. Setting the
    # field's merge rule to mean will produce the mean of slope for overlapped polygon.
    SlopeFieldIndex = fieldmappings.findFieldMapIndex("slope")
    fieldmap = fieldmappings.getFieldMap(SlopeFieldIndex)
    # Get the output field's properties as a field object
    field = fieldmap.outputField
    # Rename the field and pass the updated field object back into the field map
    field.name = "mean_slope"
    field.aliasName = "mean_slope"
    fieldmap.outputField = field
    # Set the merge rule to "mean" and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmap.mergeRule = "mean"
    fieldmappings.replaceFieldMap(SlopeFieldIndex, fieldmap)
    
    ### Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures3, joinFeatures3, outfc3, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"INTERSECT")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Calculate density

In [11]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    
    # Set local variables
    inTable = "Census2010_Shimane2BusInterSlope"
    fieldName1 = "BusDen"
    fieldName2 = "InterDen"
    fieldName3 = "ResiDen"
    expression1 = "(!Join_Count!/!AreaRes_1!)"
    expression2 = "(!Join_Count_1!/!AreaRes_1!)"
    expression3 = "(!TOTHH_H22!/!AreaRes_1!)"

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName3, "DOUBLE")

    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName3, expression3, "PYTHON_9.3")
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


## Create walkability index - Individual level
### 1. Residential density (od cost matrix to KIHON_Chugoku2010)
### 2. Bus stop density (od cost matrix to bus stop point data 2010)
### 3. Intersection density (od cost matrix to intersection with >= 3 legs point data 2012)
### 4. Slope (spatial join with mean slope 5 mesh data 2015)
### 5. Age-specific population density (od cost matrix to census point data 2010)

In [ ]:
# Residential density

In [2]:
import arcpy
from arcpy import env
import os

try:
       
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll"
    env.workspace = os.path.join(output_dir, "Walkability_calculation.gdb")
    env.overwriteOutput = True

    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2012島根\道路網_島根県版\ADF2012_DISSOLVE.gdb"
    input_gdb2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    input_gdb3 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkabiliy_Indices.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb2, "cohreAllGeocoded")
    destinations = os.path.join(input_gdb3, "KIHON_Chugoku2010")
    output_features = "RW1_res"

    #Define some OD cost matrix analysis settings
    layer_name = "RW1_res"
    travel_mode = "Driving Time"
    cut_off = "30"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)
    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the ID fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "cohreID", "TEXT")
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["cohreID"].mappedFieldName = "OBJECTID"
   
    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"Resi", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["Resi"].mappedFieldName = "HOUSEHOLDS_H22"
  
    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "coreID")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "Resi")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Weight the residential units

In [3]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    
    # Set local variables
    inTable = "ODLines1"
    fieldName1 = "Resi_w"
    expression = "w(!Total_所要時間（自動車）!,!Resi!)"
    codeblock1 = """def w(f1,f2):
        if f1 < 10:
            return f2
        elif (f1 >= 10 and f1 < 30):
            return f2 * ((30-f1)/(30-10))**1.5"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Summary statisitcs by ID

In [4]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    # Set local variables
    inFeatures = "ODLines1"
    outStatsTable = "RW1_res"
    statsFields = [["Resi_w","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Bus Stop

In [5]:
import arcpy
from arcpy import env
import os

try:
       
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll"
    env.workspace = os.path.join(output_dir, "Walkability_calculation.gdb")
    env.overwriteOutput = True

    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2012島根\道路網_島根県版\ADF2012_DISSOLVE.gdb"
    input_gdb2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    input_gdb3 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkabiliy_Indices.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb2, "cohreAllGeocoded")
    destinations = os.path.join(input_gdb3, "busstop_Chugoku2010")
    output_features = "RW2_bus"

    #Define some OD cost matrix analysis settings
    layer_name = "RW2_bus"
    travel_mode = "Driving Time"
    cut_off = "30"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)
    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the ID fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "cohreID", "TEXT")
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["cohreID"].mappedFieldName = "OBJECTID"
   
    #Load the cohre samples as origin
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)
    #Load the bus stop as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "coreID")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Weight the bus stop

In [6]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    
    # Set local variables
    inTable = "ODLines2"
    fieldName1 = "Bus_w"
    expression = "w(!Total_所要時間（自動車）!)"
    codeblock1 = """def w(f1):
        if f1 < 10:
            return 1
        elif (f1 >= 10 and f1 < 30):
            return 1 * ((30-f1)/(30-10))**1.5"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Summary statistic by ID

In [7]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    # Set local variables
    inFeatures = "ODLines2"
    outStatsTable = "RW2_bus"
    statsFields = [["Bus_w","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Intersection

In [8]:
import arcpy
from arcpy import env
import os

try:
       
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll"
    env.workspace = os.path.join(output_dir, "Walkability_calculation.gdb")
    env.overwriteOutput = True

    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2012島根\道路網_島根県版\ADF2012_DISSOLVE.gdb"
    input_gdb2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    input_gdb3 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkabiliy_Indices.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb2, "cohreAllGeocoded")
    destinations = os.path.join(input_gdb3, "Intersect_Shimane2012")
    output_features = "RW3_inter"

    #Define some OD cost matrix analysis settings
    layer_name = "RW3_inter"
    travel_mode = "Driving Time"
    cut_off = "30"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)
    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the ID fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "cohreID", "TEXT")
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["cohreID"].mappedFieldName = "OBJECTID"
   
    #Load the cohre samples as origin
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)
    #Load the bus stop as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "coreID")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [9]:
# Weight the intersection

In [11]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    
    # Set local variables
    inTable = "ODLines3"
    fieldName1 = "Inter_w"
    expression = "w(!Total_所要時間（自動車）!)"
    codeblock1 = """def w(f1):
        if f1 < 10:
            return 1
        elif (f1 >= 10 and f1 < 30):
            return 1 * ((30-f1)/(30-10))**1.5"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Summary statistics by ID

In [12]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    # Set local variables
    inFeatures = "ODLines3"
    outStatsTable = "RW3_inter"
    statsFields = [["Inter_w","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Slope

In [13]:
import arcpy
from arcpy import env
import os

try:
       
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll"
    env.workspace = os.path.join(output_dir, "Walkability_calculation.gdb")
    env.overwriteOutput = True

    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2012島根\道路網_島根県版\ADF2012_DISSOLVE.gdb"
    input_gdb2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    input_gdb3 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkabiliy_Indices.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb2, "cohreAllGeocoded")
    destinations = os.path.join(input_gdb3, "SlopeShimaneCentroid")
    output_features = "RW4_slope"

    #Define some OD cost matrix analysis settings
    layer_name = "RW4_slope"
    travel_mode = "Driving Time"
    cut_off = "30"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)
    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the ID fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "cohreID", "TEXT")
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["cohreID"].mappedFieldName = "OBJECTID"
   
    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"Slope", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["Slope"].mappedFieldName = "slope"
  
    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "coreID")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "Slope")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Weight the slope

In [14]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    
    # Set local variables
    inTable = "ODLines4"
    fieldName1 = "Slope_w"
    expression = "w(!Total_所要時間（自動車）!,!Slope!)"
    codeblock1 = """def w(f1,f2):
        if f1 < 10:
            return f2
        elif (f1 >= 10 and f1 < 30):
            return f2 * ((30-f1)/(30-10))**1.5"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Summary statistics by ID

In [15]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    # Set local variables
    inFeatures = "ODLines4"
    outStatsTable = "RW4_slope"
    statsFields = [["Slope_w","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Compute mean slope
# Calculate manually

In [ ]:
# Join all tables

In [22]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    env.overwriteOutput = True
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    inWorkspace3 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_final.gdb"
    inFeatures = os.path.join(inWorkspace1,"cohreAllGeocoded")
    joinField = "OBJECTID"
    joinTable1 = os.path.join(inWorkspace2,"RW1_res")
    joinTable2 = os.path.join(inWorkspace2,"RW2_bus")
    joinTable3 = os.path.join(inWorkspace2,"RW3_inter")
    joinTable4 = os.path.join(inWorkspace2,"RW4_slope")
    joinField2 = "OriginID"
    fieldList1 = ["SUM_Resi_w"] 
    fieldList2 = ["SUM_Bus_w"] 
    fieldList3 = ["SUM_Inter_w"] 
    fieldList4 = ["SUM_Slope_w_mean"] 
    outFeatures1 = os.path.join(inWorkspace3,"cohreRW1")
    outFeatures2 = os.path.join(inWorkspace3,"cohreRW12")
    outFeatures3 = os.path.join(inWorkspace3,"cohreRW123")
    outFeatures4 = os.path.join(inWorkspace3,"cohreRW1234")

    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField, joinTable1, joinField2, fieldList1)
    arcpy.CopyFeatures_management(inFeatures, outFeatures1)
    arcpy.JoinField_management(outFeatures1, joinField, joinTable2, joinField2, fieldList2)
    arcpy.CopyFeatures_management(outFeatures1, outFeatures2)
    arcpy.JoinField_management(outFeatures2, joinField, joinTable3, joinField2, fieldList3)
    arcpy.CopyFeatures_management(outFeatures2, outFeatures3)
    arcpy.JoinField_management(outFeatures3, joinField, joinTable4, joinField2, fieldList4)
    arcpy.CopyFeatures_management(outFeatures3, outFeatures4)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Traditional approach: 400m 800m buffer

In [ ]:
# completed manually

# Spatial join neighborhoood walkability and geocoded cohre data

In [25]:
### Import system modules
import arcpy
import os

try:
    
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_calculation.gdb"
    env.overwriteOutput = True
    
    # Set local variables
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_final.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    outWorkdspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_final.gdb"
    targetFeatures = os.path.join(inWorkspace1, "cohreRW1234")
    joinFeatures = os.path.join(inWorkspace2, "Census2010_Shimane2BusInterSlope")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkdspace, "cohreRWfinal")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
       
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("Join_Count")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("TARGET_FID")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("Join_Count_1")
    fieldmappings.removeFieldMap(z)
    b = fieldmappings.findFieldMapIndex("TARGET_FID_1")
    fieldmappings.removeFieldMap(b)
    c = fieldmappings.findFieldMapIndex("Join_Count_12")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("TARGET_FID_12")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("P11_001")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("P11_002")
    fieldmappings.removeFieldMap(f)
    
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Export data

In [26]:
# Import system modules
import arcpy
from arcpy import env

try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_final.gdb"
    # Set local variables
    inTable = "cohreRWfinal"
    outLocation = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_final.gdb"
    outTable = "cohreRWfinalTable"
    # Execute TableToTable
    arcpy.TableToTable_conversion(inTable, outLocation, outTable)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

An error occurred on line 15
実行できません。パラメーターが不正です。
ERROR 000732: Input Table: データセット cohreRWfinalTable が存在しないか、サポートされていません
(TableToExcel) を実行できませんでした。



In [27]:
# Import system modules
import arcpy
from arcpy import env

try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\Walkability_final.gdb"
    # Set local variables
    inTable = "cohreRWfinalTable"
    out_xls = r"G:\cohreRWfinal.xls"
    
    arcpy.TableToExcel_conversion(inTable, out_xls)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Create Kyuson - Cho aza address list
### 1. Spatial join with Kyuson polygon and Cho aza polygon by retaining the address name
### 2. Export the table as excel table

In [8]:
### Import system modules
import arcpy
import os
from arcpy import env

try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\CoHREAll\CoHREAll.gdb"
    env.overwriteOutput = True
    # Set local variables
    
    # Want to join PHCCover polygon to KIHON_Chugoku2010 and calculate
    # the total number of population that service area (10, 20, 30, 60 min) can cover
    targetFeatures = "Census2010Centroid"
    joinFeatures = "Kyuson_H12foAdd"
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = "Aza_KyusonAdderss"
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "","WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))
    

Script completed successfully
